In [8]:
import os
import sys
import plotly.express as px
import torch as t
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import numpy as np
import einops
from jaxtyping import Int, Float
from typing import List, Optional, Tuple
from typing_extensions import Literal
import functools
from tqdm import tqdm
from IPython.display import display
import webbrowser
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
import circuitsvis as cv
import openai
from dotenv import load_dotenv
import random
from IPython.display import display, HTML


load_dotenv()

# Set API Keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
assert OPENAI_API_KEY, "OPENAI_API_KEY environment variable is missing from .env"
openai.api_key = OPENAI_API_KEY

content = "You are a helpful mechanistic interpretability researcher who is an expert in analyzing attention patterns"
# response = openai.ChatCompletion.create(model=self.model, messages=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": prompt}], max_tokens=100, tools=tools) # tools=tools
#         response_message = response['choices'][0]['message']

In [4]:
openai_model = "gpt-3.5-turbo-16k" # "gpt-3.5-turbo"

In [2]:
from datasets import load_dataset
dataset = load_dataset("stas/openwebtext-10k", split="train", trust_remote_code=True)

In [3]:
# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

device = t.device("cuda" if t.cuda.is_available() else "mps")

In [4]:
text = dataset['text']

In [5]:
cfg = HookedTransformerConfig(
    d_model=768,
    d_head=64,
    n_heads=12,
    n_layers=2,
    n_ctx=2048,
    d_vocab=50278,
    attention_dir="causal",
    attn_only=True, # defaults to False
    tokenizer_name="EleutherAI/gpt-neox-20b", 
    seed=398,
    use_attn_result=True,
    normalization_type=None, # defaults to "LN", i.e. layernorm with weights & biases
    positional_embedding_type="shortformer"
)

weights_dir = "test-heads/attn_only_2L_half.pth"

model = HookedTransformer(cfg)
pretrained_weights = t.load(weights_dir, map_location=device)
model.load_state_dict(pretrained_weights)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

In [16]:
# model.tokenizer(text, return_tensors="pt")["input_ids"].shape
random_sentences = ["From around 1242, he was co-ruler with his father, and his relationship with some prominent aristocrats grew tense.", 
                    "Lack of approval for a congressional delegation resulted in the ceremony being delayed from the scheduled time on 14 January to the very early morning of 15 January", 
                    "He came to Sweden with Cardinal Nicholas Breakspeare in 1153 and was most likely designated to be the new Archbishop of Uppsala, but the independent church province of Sweden could only be established in 1164 after the civil war, and Henry would have been sent to organize the Church in Finland, where Christians had already existed for two centuries.", 
                    "Bright vixens jump over the moon; dozy fowl quack at the shimmering lake.", 
                    "Sphinx of black quartz, judge my vow as I stand in the heart of the Egyptian desert."]
input_ids = model.tokenizer(random_sentences, padding=True, return_tensors="pt")["input_ids"]
input_ids.shape


torch.Size([5, 70])

In [7]:
# text = "We think that powerful, significantly superhuman machine intelligence is more likely than not to be created this century. If current machine learning techniques were scaled up to this level, we think they would by default produce systems that are deceptive or manipulative, and that no solid plans are known for how to avoid this."

# logits, cache = model.run_with_cache(text, remove_batch_dim=True)

# str_tokens = model.to_str_tokens(text)
# for layer in range(model.cfg.n_layers):
#     attention_pattern = cache["pattern", layer]
#     display(cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern))

In [26]:
# logits, cache = model.run_with_cache(input_ids)
random_idx = 0
# new_text = random_sentences[random_idx]

# str_tokens = model.to_str_tokens(new_text)
for layer in range(model.cfg.n_layers):
    attention_pattern = auto_attention.cache["pattern", 0][random_idx, ...]
    display(cv.attention.attention_patterns(tokens=auto_attention.str_tokens[0], attention=attention_pattern))

In [ ]:
def gini_coefficient(x):
    x = x.flatten() #all values are treated equally, arrays must be 1d
    n = x.shape[0]
    x = x.sort()[0] #values must be sorted
    index = t.arange(1, n+1, device=device) #index per array element
    return (t.sum((2 * index - n  - 1) * x)) / (n * t.sum(x)) #Gini coefficient 

In [10]:
def get_random_row_idx(end_idx: int) -> int:
    min_idx = 10 if end_idx > 10 else 1
    max_idx = end_idx
    return random.randint(min_idx, max_idx)

In [21]:
# NEWEST
prompt_prefix = """
Here is a row by row break down of which tokens were attended to for each individual generation step for an autoregressive transformer.
Each row is labeled with its index and is of the format for each row: `\{text\}\\n\{{token} {multiple} for token, multiple in above_avg_tokens}`.
Notably, the tokens with above average attention are sorted by their score's multiple relative to the average attention score for that row,
and this multiple is included next to it, i.e. `tokenX 2, tokenY 1.5` etc.
"""

prompt_suffix = """
Please generate a response in the format `row \{idx\}: \{your generated description of this row's pattern of attention here\}` for each row.
Be as specific as possible, making sure to consider all possible reasons certain tokens may have been attended to more than others, including 
the position of the tokens, what the tokens themselves represent, the attention scores, and how these all interact.
Limit your description to 1-3 sentences per row.
"""

num_row_samples = 1

def get_multiple(score, avg_score):
    return round((score / avg_score).item(), 1)

def get_attention_pattern_prompt_for_rows(cache: ActivationCache, layer: int, head: int, tokens: List[List[str]], pad_token: str) -> str:
    attention_pattern = cache["pattern", layer][:, head, ...]
    n_ctx, batch_size = len(tokens[0]), len(tokens)
    assert attention_pattern.shape == (batch_size, n_ctx, n_ctx), f"The cached attention pattern shape {attention_pattern.shape} != tokens shape {(batch_size, n_ctx, n_ctx)}"
    
    # Randomly sample up to 5 batch indices
    sampled_batch_indices = random.sample(range(batch_size), min(5, batch_size))
    
    # Generate rows of string for each sampled index
    rows = []
    for sample_idx in sampled_batch_indices:  # Iterate over sampled batches
        end_idx = n_ctx
        if pad_token in tokens[sample_idx][1:]:
            end_idx = tokens[sample_idx][1:].index(pad_token)
        idx = get_random_row_idx(end_idx)
        # Only consider tokens that the current token could attend to (itself and previous tokens)
        num_tokens = idx + 1
        relevant_tokens = tokens[sample_idx][:num_tokens]  # Adjust for batch index
        text = "".join(relevant_tokens)
        relevant_scores = attention_pattern[sample_idx, idx, :num_tokens]  # Adjust for batch index
        avg_score = 1 / num_tokens
        
        # Calculate summary statistics
        above_avg_tokens = [(token, get_multiple(score, avg_score)) for token, score in zip(relevant_tokens, relevant_scores) if score > avg_score]
        above_avg_tokens.sort(key=lambda x: x[1], reverse=True)
        above_avg_tokens = ", ".join([f"{token} {multiple}" for token, multiple in above_avg_tokens])
        
        row = f"{text}\n{above_avg_tokens}\n"
        
        rows.append(row)
    
    # Combine the sampled rows into a single string with line breaks
    rows_str = "\n".join(rows)
    
    # Combine the prefix, rows, and suffix into the final prompt
    prompt = f"{prompt_prefix}\n{rows_str}\n{prompt_suffix}"
    
    return prompt

In [70]:
# Select random sentences
selected_sentences = random.sample(text, 10)

input_ids = model.tokenizer(selected_sentences, padding=True, return_tensors="pt")["input_ids"]

max_length = 40
if input_ids.shape[1] > max_length:
    input_ids = input_ids[:, :max_length]

decoded = model.tokenizer.batch_decode(input_ids)
str_tokens = model.to_str_tokens(decoded, prepend_bos=False)

In [71]:
_, new_cache = model.run_with_cache(input_ids)

In [73]:
prompt = get_attention_pattern_prompt_for_rows(new_cache, 0, 2, str_tokens, model.tokenizer.pad_token)
print(prompt)


Here is a row by row break down of which tokens were attended to for each individual generation step for an autoregressive transformer.
Each row is labeled with its index and is of the format for each row: `\{text\}\n\{{token} {multiple} for token, multiple in above_avg_tokens}`.
Notably, the tokens with above average attention are sorted by their score's multiple relative to the average attention score for that row,
and this multiple is included next to it, i.e. `tokenX 2, tokenY 1.5` etc.

Truman Tells Stalin, July 24, 1945

Most of
 Stalin 7.4,  1945 2.4

MLB Hall of Fame catcher Ivan "Pudge" Rodriguez stands with Texas Rangers manager Jeff Banister (28) in the dugout before a spring
ML 21.7,  before 1.4,  in 1.0

NSA whistleblower Edward Snowden appeared via video link from Moscow at the CeBIT IT trade conference in
 Moscow 5.0,  Snowden 2.8,  conference 2.2,  trade 1.9, blower 1.5,  Edward 1.3, N 1.1

Universal's film will be tied to the Caribbean area where several ships and pla

In [32]:
# num_samples = 5
# random_indices = random.sample(range(len(sentences)), num_samples)
# random_sentences = [sentences[i] for i in random_indices]
# random_sentences_str_toks = [model.to_str_tokens(sentence) for sentence in random_sentences]
# print(random_sentences)
# print(random_sentences_str_toks)


# selected_sentences = random.sample(list(sentences), 5)
# sentences
# input_ids = model.tokenizer(selected_sentences, padding=True, return_tensors="pt")["input_ids"]
# print(input_ids)
# str_tokens = [model.to_str_tokens(sentence) for sentence in input_ids]
# prompt = get_attention_pattern_prompt_for_rows(cache, 0, 0, str_tokens, model.tokenizer.pad_token)
# print(prompt)


[['I', 'said', '"', 'What', 'for', '?"\''], ["'", 'We', 'called', 'him', 'Tortoise', 'because', 'he', 'taught', 'us', ",'", 'said', 'the', 'Mock', 'Turtle', 'angrily', ':', "'", 'really', 'you', 'are', 'very', 'dull', "!'"], ['cried', 'the', 'Gryphon', ',', 'and', ',', 'taking', 'Alice', 'by', 'the', 'hand', ',', 'it', 'hurried', 'off', ',', 'without', 'waiting', 'for', 'the', 'end', 'of', 'the', 'song', '.'], ["'", 'I', "'", 'm', 'glad', 'they', "'", 've', 'begun', 'asking', 'riddles', '.--', 'I', 'believe', 'I', 'can', 'guess', 'that', ",'", 'she', 'added', 'aloud', '.'], ["'", 'And', 'ever', 'since', 'that', ",'", 'the', 'Hatter', 'went', 'on', 'in', 'a', 'mournful', 'tone', ',', "'", 'he', 'won', "'", 't', 'do', 'a', 'thing', 'I', 'ask', '!']]
[[['<|endoftext|>', 'I'], ['<|endoftext|>', 'said'], ['<|endoftext|>', '"'], ['<|endoftext|>', 'What'], ['<|endoftext|>', 'for'], ['<|endoftext|>', '?"', "'"]], [['<|endoftext|>', "'"], ['<|endoftext|>', 'We'], ['<|endoftext|>', 'called'], ['

In [ ]:
input_ids = model.tokenizer(random_sentences, padding=True, return_tensors="pt")["input_ids"]
str_tokens = [model.to_str_tokens(sentence) for sentence in input_ids]
prompt = get_attention_pattern_prompt_for_rows(cache, 0, 0, str_tokens, model.tokenizer.pad_token)
print(prompt)

In [19]:
input_ids = model.tokenizer(random_sentences, padding=True, return_tensors="pt")["input_ids"]
tokenized_sentences = model.to_str_tokens(input_ids)
tokenized_sentences

AssertionError: Invalid tokens input to to_str_tokens, has shape: torch.Size([5, 70])

row 59: The attention is spread across multiple tokens, mostly focusing on the words discussing the potential creation of powerful machine intelligence and the consequences of scaling up current machine learning techniques. The highest attention scores are observed for tokens related to the idea that systems would be deceptive or manipulative without solid plans to avoid such behavior.

row 25: Attention spans across various tokens, particularly focusing on the discussion of the likelihood of powerful machine intelligence being created in this century. Attention weights are highest for tokens related to the possibility of current machine learning techniques leading to this outcome.

row 27: Attention mostly concentrates on tokens related to the discussion of powerful machine intelligence being created and the assessment of the likelihood of this occurrence. The highest attention scores are observed for tokens discussing the consequences of scaling up current machine learning techniques

In [58]:
a = model.tokenizer("<|endoftext|>(3.78x),  than(3.46x),  to(2.08x),  be(1.85x),  this(1.66x),  not(1.27x),  is(1.13x)")
b = a["input_ids"]
print(len(b), b)

55 [0, 9, 20, 15, 3141, 89, 582, 50276, 14644, 9, 20, 15, 2950, 89, 582, 50276, 936, 9, 19, 15, 2904, 89, 582, 50276, 1257, 9, 18, 15, 2227, 89, 582, 50276, 2520, 9, 18, 15, 2526, 89, 582, 50276, 1439, 9, 18, 15, 1630, 89, 582, 50276, 261, 9, 18, 15, 1012, 89, 10]


In [30]:
class HeadData:
    def __init__(self):
        self.descriptions = []
        self.samples = None
        self.ranked_multiples = None
        

class AutoAttention:
    def __init__(self, 
                 model: HookedTransformer, 
                 openai_model: Literal["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-4", "gpt-4-32k"],
                 openai_api_key: str,
                 text: List[List[str]],
                 min_length = 10,
                 max_length = 30
                 ) -> None:
        self.model = model
        self.openai_model = openai_model
        self.OPENAI_API_KEY = openai_api_key
        self.heads = {layer_idx: {head_idx: HeadData() for head_idx in range(self.model.cfg.n_heads)} for layer_idx in range(self.model.cfg.n_layers)}
        self.min_length = min_length
        self.max_length = max_length
        self.text = text
        self.str_tokens = None
        self.input_ids = None
        self.cache = None
        self.pad_token_id = model.tokenizer.pad_token_id
        self.prompt_prefix = ("Here is a row by row break down of which tokens were attended to for each individual generation step for a transformer. "
                              "Each row is labeled with its index and is of the format for each row: `{text}\\n{{token} {multiple} for token, multiple in above_avg_tokens}`. "
                              "Notably, the tokens with above average attention are sorted by their score's multiple relative to the average attention score for that row, "
                              "and this multiple is included next to it, i.e. `tokenX 2, tokenY 1.5` etc.")
        self.prompt_suffix = ("Please generate a response that summarizes what this attention head pays attention to, "
                             "generalizing from the specific contexts for each example. Be as specific as possible, considering all possible "
                             "reasons certain tokens may have been attended to more than others, including the position of "
                             "the tokens, what the tokens themselves represent, the attention scores, and how these all interact. "
                             "Do NOT explain how attention heads or transformers work or include highly general information. "
                             "Limit your response to 1 paragraph.")
    
    def _create_cache(self):
        input_features = model.tokenizer(self.text, padding=True, return_tensors="pt")
        input_ids = input_features["input_ids"]
        if input_ids.shape[1] > self.max_length:
            input_ids = input_ids[:, :self.max_length]
        decoded = model.tokenizer.batch_decode(input_ids)
        str_tokens = model.to_str_tokens(decoded, prepend_bos=False)
        _, cache = model.run_with_cache(input_ids)
        
        self.str_tokens = str_tokens
        self.input_ids = input_ids
        self.cache = cache
        
    def create_samples(self, head=0, layer=0):
        print(f"Creating new samples for layer {layer} head {head}")
        if self.cache is None:
            self._create_cache()
        # Calculate the average score multiples
        attention_patterns = self.cache["pattern", layer][:, head, ...]
        avg_scores = 1 / t.arange(1, attention_patterns.shape[1] + 1).float().to(attention_patterns.device)
        avg_scores = avg_scores.unsqueeze(-1)
        attention_multiples = attention_patterns / avg_scores
        
        # Create the samples of [((seq_idx, length_idx), score_multiples1), ...]
        samples = []
        for seq_idx in range(attention_patterns.shape[0]):
            seq_ids = self.input_ids[seq_idx].tolist()
            pad_token_idx = seq_ids.index(self.pad_token_id) if self.pad_token_id in seq_ids else float('inf')
            for length_idx in range(self.min_length, min(pad_token_idx, attention_patterns.shape[1])):
                if length_idx >= pad_token_idx: 
                    break
                str_tokens = self.str_tokens[seq_idx][:length_idx+1]
                score_multiples = attention_multiples[seq_idx, length_idx, :length_idx+1].tolist()
                score_multiples = [(str_token, round(multiple, 1)) for str_token, multiple in zip(str_tokens, score_multiples) if multiple > 1]
                score_multiples.sort(key=lambda x: x[1])
                samples.append(((seq_idx, length_idx), score_multiples))

        head_data = self.heads[layer][head]
        head_data.samples = samples
        head_data.ranked_multiples = None
    
    def get_head_samples(self, head=0, layer=0, num_samples=10):
        head_data = self.heads[layer][head]
        if head_data.samples is None:
            self.create_samples(head=head, layer=layer)
        samples = head_data.samples
        sample_indices = random.sample(range(len(samples)), num_samples)
        rows = []
        for i in sample_indices:
            (seq_idx, length_idx), score_multiples = samples[i]
            token_ids = self.input_ids[seq_idx][:length_idx+1]
            text = self.model.tokenizer.decode(token_ids)
            above_avg_tokens = ", ".join([f"{token} {multiple}" for token, multiple in score_multiples])
            row = f"{text}\n{above_avg_tokens}\n"
            rows.append(row)
        
        return "\n".join(rows)
    
    def describe_head(self, head=0, layer=0, num_samples=10, custom_prompt=None):
        # Retrieve samples
        head_data = self.heads[layer][head]
        if head_data.samples is None:
            self.create_samples(head=head, layer=layer)
        samples = self.get_head_samples(head=head, layer=layer, num_samples=num_samples)
        
        # Create prompt
        prompt = f"{self.prompt_prefix}\n{samples}"
        if custom_prompt:
            prompt += f"\n{custom_prompt}"
        prompt += f"\n{self.prompt_suffix}"
        
        # Send prompt and save result
        print(f"Making API call to {self.openai_model}...")
        description = self._prompt_gpt(prompt)
        head_data.descriptions.append(description)
        return prompt, description
        
    
    def _prompt_gpt(self, prompt: str) -> str:
        content = "You are a helpful mechanistic interpretability researcher who is an expert in analyzing attention patterns"
        response = openai.ChatCompletion.create(model=self.openai_model, messages=[{"role": "system", "content": content}, {"role": "user", "content": prompt}])
        response_content = response['choices'][0]['message']['content']
        return response_content
    
    def _create_ranked_multiples(self, layer=0, head=0):
        head_data = self.heads[layer][head]
        if head_data.samples is None:
            self.create_samples(head=head, layer=layer)
    
        ranked_multiples = []
        for sample in head_data.samples:
            (seq_idx, length_idx), multiples = sample
            for (token, multiple) in multiples:
                str_toks = self.str_tokens[seq_idx][:length_idx+1]
                attention_pattern = self.cache["pattern", layer][seq_idx, head, length_idx, :length_idx+1]
                ranking = [token, multiple, str_toks, attention_pattern]
                ranked_multiples.append(ranking)
                
        ranked_multiples.sort(key=lambda x: x[1], reverse=True)
        head_data.ranked_multiples = ranked_multiples
    
    def get_ranked_multiples(self, layer=0, head=0, str_token=None, num_multiples=10, reverse=False, display=False):
        head_data = self.heads[layer][head]
        if head_data.ranked_multiples is None:
            self._create_ranked_multiples(layer=layer, head=head)
        if reverse:
            multiples = head_data.ranked_multiples[-num_multiples:]
        else:
            multiples = head_data.ranked_multiples[:num_multiples]
        
        # Optionally only retrieve certain tokens
        if str_token is not None:
            count = 0
            multiples = []
            for multiple in head_data.ranked_multiples:
                ranked_str_token, *_ = multiple
                if str_token == ranked_str_token:
                    multiples.append(multiple)
                    count += 1
                if count >= num_multiples:
                    break
        
        if display:
            title = f"Layer {layer} Head {head}, Top {num_multiples} / {len(head_data.ranked_multiples)} Multiples"
            self._display_rows(multiples)
        return multiples
    
    def get_random_multiples(self, layer=0, head=0, num_multiples=10, display=False):
        head_data = self.heads[layer][head]
        if head_data.ranked_multiples is None:
            self._create_ranked_multiples(layer=layer, head=head)
        
        indices = random.sample(range(len(head_data.ranked_multiples)), num_multiples)
        random_multiples = [head_data.ranked_multiples[i] for i in indices]
        
        if display: 
            title = f"Layer {layer} Head {head}, {num_multiples} / {len(head_data.ranked_multiples)} Random Multiples"
            self._display_rows(random_multiples, title=title)
        return random_multiples
    
    def _display_rows(self, rows, title=""):
        html_str = "<table style='border:1px solid black;'>"
        # Add title row if title is not empty
        if title:
            html_str += f"<tr><th colspan='10' style='text-align:center'>{title}</th></tr>"
        # Add column headers
        html_str += "<tr><th>Token</th><th>Multiple of Avg. score</th><th>Pattern</th></tr>"
        for token, multiple, str_tokens, attention_pattern in rows:
            html_str += "<tr>"
            html_str += f"<td style='border:1px solid black;'>{token}</td>"
            html_str += f"<td style='border:1px solid black;'>{multiple}</td>"

            # Add pattern cells
            for i, p in enumerate(attention_pattern.tolist()):
                # Calculate color intensity based on pattern value (assuming pattern value is between 0 and 1)
                intensity = int(p * 255)
                html_str += f"<td style='border:1px solid black; background-color:rgb({intensity}, 0, 0);'>{str_tokens[i].strip()}</td>"  # Changed color to red
            html_str += "</tr>"
        html_str += "</table>"
        display(HTML(html_str))


auto_attention = AutoAttention(model=model, 
                               text=text[:100], 
                               openai_model="gpt-3.5-turbo-1106", 
                               openai_api_key=OPENAI_API_KEY
                               )

In [190]:
# auto_attention.heads[0][0].samples[10]

ranked_multiples = auto_attention.get_ranked_multiples(num_multiples=10)

Creating new samples for layer 0 head 0


In [33]:
test = auto_attention.get_random_multiples(num_multiples=20, display=True)

In [31]:
a = auto_attention.get_ranked_multiples(layer=0, head=3, reverse=True, num_multiples=100, display=False)
for i, j, *_ in a:
    print(i, j)

Creating new samples for layer 0 head 3

 1.0

 1.0

 1.0

 1.0

 1.0

 1.0

 1.0

 1.0

 1.0
? 1.0

 1.0

 1.0
? 1.0
? 1.0
 He 1.0

 1.0
! 1.0
 TO 1.0
! 1.0
 our 1.0
! 1.0
 Hopkins 1.0
 — 1.0
 of 1.0
 Princess 1.0
 Princess 1.0
 Pokémon 1.0
 ( 1.0
 A 1.0
 couples 1.0
 couples 1.0
 Chelsea 1.0
 to 1.0
 London 1.0
 about 1.0
: 1.0
 Prince 1.0
Monday 1.0
Monday 1.0
Monday 1.0
Monday 1.0
Monday 1.0
 it 1.0

 1.0
 He 1.0
 He 1.0
 character 1.0
s 1.0
 “ 1.0
 Europe 1.0
 ship 1.0
 ship 1.0
 in 1.0
 his 1.0
? 1.0
 killed 1.0
’ 1.0
 shit 1.0
 shit 1.0
 of 1.0
— 1.0
— 1.0
. 1.0
" 1.0
" 1.0
 of 1.0
 in 1.0
 in 1.0
 Italian 1.0
 vessel 1.0
 directors 1.0
 The 1.0
 artists 1.0
, 1.0
 but 1.0
 McCain 1.0
 it 1.0
 by 1.0
 in 1.0
 it 1.0
 The 1.0
 Solid 1.0
: 1.0
. 1.0

 1.0
— 1.0
 October 1.0
 October 1.0
 White 1.0

 1.0

 1.0

 1.0
s 1.0
 3 1.0
 on 1.0
 drunk 1.0
 drunk 1.0
 drunk 1.0

 1.0
 drunk 1.0


Calc multiples in tensors, then move to python for storing with str_tokens

How to store "max activating" examples? I want to be able to retrieve the original text, I want to be able to see it on a per token basis, and I want to be able to see it on an absolute magnitude basis. Might have to create a new list on the fly and sort it?
Eh, just recalculate every time. Those might be kind of fun to see the results for, and you could visualize it.

No gini coefficient though.